In [ ]:
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.linear_model import BayesianRidge
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import ExtraTreeClassifier
from sklearn import linear_model
import lightgbm
import catboost
import xgboost
from sklearn.metrics import r2_score
%matplotlib
###########wrapping root mean square error for later calls##########
def compute_mae_mse_rmse(target,prediction):
    error = []
    for i in range(len(target)):
        error.append(target[i] - prediction[i])
    squaredError = []
    absError = []
    for val in error:
        squaredError.append(val * val)  # target-prediction之差平方
        absError.append(abs(val))  # 误差绝对值
    mae=sum(absError)/len(absError)  # 平均绝对误差MAE
    mse=sum(squaredError)/len(squaredError)  # 均方误差MSE
    RMSE=sum(absError)/len(absError)
    R2=r2_score(target,prediction)
    return mae,mse,RMSE,R2
###########loading data##########
fdata=pd.read_csv('database_filled.csv',encoding="gbk")
raw_data=fdata.loc[:,[
                      'Fe Cotent (wt. %)',#2
                      'O Content (at. %)',#4
                      'Content of Pyridinic Species (at. %)',#5
                      'Raman ID/IG Ratio',#8
                      'BET Surface Area (m2 g-1)',#9
                      'Micropore Ratio',#10
                      'Mesopore Ratio',#11
                      'Catalyst Loading on RDE/RRDE(mg cm-2)',#12
                      'Half Wave Potential in Acid Condition (vs. RHE)'#the classification target#7
                        ]]
###########train test splitting##########
raw_param=raw_data.iloc[:,0:8]
raw_power=raw_data.iloc[:,8]

In [ ]:
def gridsearch(model,param,algorithm_name):
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    result = best_model.predict(X_test)
    x_prediction_07=result
    y_real_07=y_test_values
    x_prediction_07_series=pd.Series(x_prediction_07)
    y_real_07_series=pd.Series(y_real_07)
    ###########evaluating the regression quality##########
    corr_ann = round(x_prediction_07_series.corr(y_real_07_series), 5)
    error_val= compute_mae_mse_rmse(x_prediction_07,y_real_07)
    rank = [index for index, value in sorted(list(enumerate(best_model.feature_importances_)), key=lambda x:x[1],reverse=True)]
    print(algorithm_name)
    print(best_model.feature_importances_)
    print('Best Regressor:',grid.best_params_,'Best Score:', grid.best_score_)
    print(error_val)
    print(corr_ann)
    x_y_x=np.arange(0.5,1,0.1)
    x_y_y=np.arange(0.5,1,0.1)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.scatter(x_prediction_07,y_real_07,color='red',label=algorithm_name)
    ax.plot(x_y_x,x_y_y)
    plt.legend()
    plt.xlabel(u"Predicted_Half_Wave_Potential V (vs. RHE)")
    plt.ylabel(u"Real_Half_Wave_Potential V (vs. RHE)")
    plt.savefig('%s scatters.png' %algorithm_name)
    fig2 = plt.figure()
    name_list=[   
                  'Fe Cotent (wt. %)',#2
                      'O Content (at. %)',#4
                      'Content of Pyridinic Species (at. %)',#5
                      'Raman ID/IG Ratio',#8
                      'BET Surface Area (m2 g-1)',#9
                      'Micropore Ratio',#10
                      'Mesopore Ratio',#11
                      'Catalyst Loading on RDE/RRDE(mg cm-2)',#12
    ]
    plt.bar(range(len(best_model.feature_importances_)), best_model.feature_importances_,color='rgb',tick_label=name_list)
    plt.xticks(rotation=90)
    plt.savefig('%s importance.png' %algorithm_name)
    plt.show()
    print('finished')

In [ ]:
seed=71
X_train, X_test, y_train, y_test = train_test_split(raw_param, raw_power, test_size=.15,random_state=seed)
y_test_values=y_test.values.astype(np.float32)

##########LGBM gridsearch CV for best hyperparameter##########
model_LightGBMRegressor=lightgbm.LGBMRegressor(random_state=1,verbose=-1)
param_light = {
    'boosting_type':['gbdt','rf'],
    'learning_rate':[0.01,0.02,0.05,0.1,0.2,1],
    'n_estimators':[100,200,400,800],
    'subsample':[0.5,0.6,0.7,0.8,0.9,1],
    'reg_alpha':[0,0.001,0.01],
    'reg_lambda':[0,0.001,0.01] 
}
gridsearch(model_LightGBMRegressor,param_light,'LightGBM')

##########XGBoost gridsearch CV for best hyperparameter##########
model_XGBRegressor=xgboost.XGBRegressor(objective='reg:squarederror',random_state=1,verbose=0)
param_xg = {
    'booster':['gbtree'],
    'learning_rate':[0.01,0.02,0.05,0.1,0.2,1],
    'n_estimators':[100,200,400,800],
    'max_depth':[5,7,9,11],
    'subsample':[0.5,0.6,0.7,0.8,0.9,1],
    'reg_alpha':[0,0.001,0.01],
    'reg_lambda':[0,0.001,0.01]
}
gridsearch(model_XGBRegressor,param_xg,'XGBoost')

##########CatBoost gridsearch CV for best hyperparameter##########
model_CatRegressor=catboost.CatBoostRegressor(random_state=1,verbose=0)
param_cat = {
    'learning_rate':[0.01,0.02,0.05,0.1,0.2,1],
    'n_estimators':[100,200,400,800],
    'max_depth':[5,7,9,11],
    'subsample':[0.5,0.6,0.7,0.8,0.9,1]
}
gridsearch(model_CatRegressor,param_cat,'CatBoost')


###########GradientBoost gridsearch CV for best hyperparameter##########
model_GradientBoostingRegressor = ensemble.GradientBoostingRegressor(random_state=1)
###########defining the parameters dictionary##########
param_GB = {
            'learning_rate':[0.01,0.02,0.05,0.1,0.2,1],
            'n_estimators':[100,200,400,800],
            'criterion':['friedman_mse','mae','mse'],
             'max_features':['auto','sqrt','log2'],
             'loss':['ls', 'lad', 'huber', 'quantile']
}
gridsearch(model_GradientBoostingRegressor,param_GB,'GradientBoost')

###########RandomForest gridsearch CV for best hyperparameter##########
model_RandomForestRegressor = ensemble.RandomForestRegressor(random_state=1)
###########defining the parameters dictionary##########
param_RF = {
                'n_estimators':[100,200,400,800],
        'criterion':['mse','mae'],
        'max_features':['auto','sqrt','log2']
}
gridsearch(model_RandomForestRegressor,param_RF,'Random Forest')


###########Extra Tree gridsearch CV for best hyperparameter##########
model_ExtraTreeRegressor = ExtraTreeRegressor(random_state=1)
param_ET = {
                   'max_depth':[5,7,9,11],
           'max_features':['auto','sqrt','log2'],
           'criterion' : ["mse", "friedman_mse", "mae"],
           'splitter' : [ "best",'random']
}
gridsearch(model_ExtraTreeRegressor,param_ET,'Extra Tree')


###########Decision Tree gridsearch CV for best hyperparameter##########
model_DecisionTreeRegressor = tree.DecisionTreeRegressor(random_state=1)
param_DT = {
                   'max_depth':[5,7,9,11],
           'max_features':['auto','sqrt','log2'],
           'criterion' : ["mse", "friedman_mse", "mae"],
           'splitter' : [ "best",'random']
}
gridsearch(model_DecisionTreeRegressor,param_DT,'Decision Tree')


###########AdaBoost gridsearch CV for best hyperparameter##########
model_AdaBoostRegressor = ensemble.AdaBoostRegressor(random_state=1)
param_Ada = {
                'n_estimators':[50,100,200,400,800],
        'learning_rate':[0.01,0.02,0.05,0.1,0.2,1],
        'loss':['linear', 'square', 'exponential']
}
gridsearch(model_AdaBoostRegressor,param_Ada,'AdaBoost')